# Project Phase 5 Appendix: Data Cleaning Description
### Benjamin Hu (bh383), Caroline Huang (ch643)

# Data Preparation

### Importing Libraries

In [1]:
## load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy import stats
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import train_test_split
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### Data Cleaning

1. For the first part, our data consists of 39 data sets. Each of these data sets have the measurements for the daily Air Quality Index (AQI) for their respective year. Each data set has the following columns: CBSA, CBSA Code, Date, AQI, Category, Defining Paramter, Defining Site and Number of Sites Reporting. We will combine all 39 of these data sets to create one dataframe with CBSA, Date and AQI because those are the most important columns to our research.

In [2]:
def create_df(file_year, drop_cols, data_name = 'daily_aqi_by_cbsa_', date_col_name = 'date'):
    '''
    param file_year: a list of ints, indicating year
    param drop_cols: a list of strings, matching column names to be dropped
    param data_name: the name of the files to be used
    param date_col: the name of the date column from the dataset
    returns the file in a form of a pd dataframe
    ''' 
    aqi_data = pd.read_csv("{}{}.csv".format(data_name, file_year))
    aqi_data.drop(drop_cols, inplace = True, axis = 1)
    aqi_data_columns = aqi_data.columns.values
    aqi_data_columns = [str.lower() for str in aqi_data_columns]
    aqi_data_columns = [str.replace(" ", "_") for str in aqi_data_columns]
    aqi_data = aqi_data.copy()
    aqi_data.columns = aqi_data_columns
    aqi_data[date_col_name] = pd.to_datetime(aqi_data[date_col_name])
    aqi_data.dropna(inplace = True)
    return aqi_data

The Training Data for AQI, union_aqi

In [3]:
# Columns that are not needed in our research are dropped to make it easier to manipulate the df because the df is already pretty big.

drop = ['CBSA Code', 'Category', 'Defining Parameter', 'Defining Site', 'Number of Sites Reporting']
years = range(1980, 2014)
union_aqi = pd.concat([create_df(file_year, drop) for file_year in years], ignore_index = True)
union_aqi['year'] = union_aqi['date'].dt.year
union_aqi['month'] = union_aqi['date'].dt.month
union_aqi['year_month'] = union_aqi['month'].map(str) + '-' + union_aqi['year'].map(str)
union_aqi['year_month'] = pd.to_datetime(union_aqi['year_month'], format='%m-%Y')
union_aqi['months_since'] = ((union_aqi['date'] - union_aqi['date'][0])/np.timedelta64(1, 'M')).astype(int)
display(union_aqi)

,cbsa,date,aqi,year,month,year_month,months_since
0,"Akron, OH",1980-01-01,112,1980,1,1980-01-01,0
1,"Akron, OH",1980-01-02,43,1980,1,1980-01-01,0
2,"Akron, OH",1980-01-03,49,1980,1,1980-01-01,0
3,"Akron, OH",1980-01-04,76,1980,1,1980-01-01,0
4,"Akron, OH",1980-01-05,89,1980,1,1980-01-01,0
...,...,...,...,...,...,...,...
4725490,"Yuma, AZ",2013-12-27,23,2013,12,2013-12-01,407
4725491,"Yuma, AZ",2013-12-28,24,2013,12,2013-12-01,407
4725492,"Yuma, AZ",2013-12-29,34,2013,12,2013-12-01,407
4725493,"Yuma, AZ",2013-12-30,27,2013,12,2013-12-01,407


The Test Data for AQI, test_aqi

In [4]:
test_years = range(2014, 2019)
test_aqi = pd.concat([create_df(file_year, drop) for file_year in test_years], ignore_index = True)
test_aqi['year'] = test_aqi['date'].dt.year
test_aqi['month'] = test_aqi['date'].dt.month
test_aqi['year_month'] = test_aqi['month'].map(str) + '-' + test_aqi['year'].map(str)
test_aqi['year_month'] = pd.to_datetime(test_aqi['year_month'], format='%m-%Y')
test_aqi['months_since'] = ((test_aqi['date'] - union_aqi['date'][0])/np.timedelta64(1, 'M')).astype(int)
display(test_aqi)

,cbsa,date,aqi,year,month,year_month,months_since
0,"Aberdeen, SD",2014-01-02,54,2014,1,2014-01-01,408
1,"Aberdeen, SD",2014-01-05,19,2014,1,2014-01-01,408
2,"Aberdeen, SD",2014-01-08,25,2014,1,2014-01-01,408
3,"Aberdeen, SD",2014-01-11,21,2014,1,2014-01-01,408
4,"Aberdeen, SD",2014-01-14,15,2014,1,2014-01-01,408
...,...,...,...,...,...,...,...
880977,"Yuma, AZ",2018-12-27,36,2018,12,2018-12-01,467
880978,"Yuma, AZ",2018-12-28,27,2018,12,2018-12-01,467
880979,"Yuma, AZ",2018-12-29,36,2018,12,2018-12-01,467
880980,"Yuma, AZ",2018-12-30,20,2018,12,2018-12-01,467


The Training Data for Temperature, union_temp

In [5]:
drop2 = ['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Pollutant Standard', 'Event Type', 'Observation Count', 'Observation Percent', 'Sample Duration', '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name', 'Local Site Name', 'Date of Last Change', 'Address', 'Units of Measure', 'State Name', 'County Name', 'City Name']
union_temp = pd.concat([create_df(file_year, drop2, 'daily_TEMP_', 'date_local') for file_year in years], ignore_index = True)
union_temp = union_temp.rename(columns = {'date_local' : 'date', 'arithmetic_mean' : 'temp'})
union_temp['year'] = union_temp['date'].dt.year
union_temp['month'] = union_temp['date'].dt.month
union_temp['year_month'] = union_temp['month'].map(str) + '-' + union_temp['year'].map(str)
union_temp['year_month'] = pd.to_datetime(union_temp['year_month'], format='%m-%Y')
union_temp['months_since'] = ((union_temp['date'] - union_temp['date'][0])/np.timedelta64(1, 'M')).astype(int)
display(union_temp)

,date,temp,cbsa_name,year,month,year_month,months_since
0,1980-01-01,47.625000,"Huntsville, AL",1980,1,1980-01-01,0
1,1980-01-02,58.727273,"Huntsville, AL",1980,1,1980-01-01,0
2,1980-01-03,56.625000,"Huntsville, AL",1980,1,1980-01-01,0
3,1980-01-04,47.875000,"Huntsville, AL",1980,1,1980-01-01,0
4,1980-01-05,41.125000,"Huntsville, AL",1980,1,1980-01-01,0
...,...,...,...,...,...,...,...
4549798,2013-12-27,35.083333,"Evanston, WY",2013,12,2013-12-01,407
4549799,2013-12-28,29.208333,"Evanston, WY",2013,12,2013-12-01,407
4549800,2013-12-29,20.250000,"Evanston, WY",2013,12,2013-12-01,407
4549801,2013-12-30,25.666667,"Evanston, WY",2013,12,2013-12-01,407


The Test Data for Temperature, test_temp

In [6]:
test_temp = pd.concat([create_df(file_year, drop2, 'daily_TEMP_', 'date_local') for file_year in test_years], ignore_index = True)
test_temp = test_temp.rename(columns = {'date_local' : 'date', 'arithmetic_mean' : 'temp'})
test_temp['year'] = test_temp['date'].dt.year
test_temp['month'] = test_temp['date'].dt.month
test_temp['year_month'] = test_temp['month'].map(str) + '-' + test_temp['year'].map(str)
test_temp['year_month'] = pd.to_datetime(test_temp['year_month'], format='%m-%Y')
test_temp['months_since'] = ((test_temp['date'] - union_temp['date'][0])/np.timedelta64(1, 'M')).astype(int)
display(test_temp)

,date,temp,cbsa_name,year,month,year_month,months_since
0,2014-01-01,42.566667,"Birmingham-Hoover, AL",2014,1,2014-01-01,408
1,2014-01-02,40.516667,"Birmingham-Hoover, AL",2014,1,2014-01-01,408
2,2014-01-03,26.825000,"Birmingham-Hoover, AL",2014,1,2014-01-01,408
3,2014-01-04,37.670833,"Birmingham-Hoover, AL",2014,1,2014-01-01,408
4,2014-01-05,44.820833,"Birmingham-Hoover, AL",2014,1,2014-01-01,408
...,...,...,...,...,...,...,...
1318589,2018-12-27,11.500000,"Evanston, WY",2018,12,2018-12-01,467
1318590,2018-12-28,8.750000,"Evanston, WY",2018,12,2018-12-01,467
1318591,2018-12-29,12.583333,"Evanston, WY",2018,12,2018-12-01,467
1318592,2018-12-30,21.000000,"Evanston, WY",2018,12,2018-12-01,467


union_temp2 dataframe

In [7]:
temp_yrs2 = range(1980, 2000)
union_temp2 = pd.concat([create_df(file_year, drop2, 'daily_TEMP_', 'date_local') for file_year in temp_yrs2], ignore_index = True)
union_temp2 = union_temp2.rename(columns = {'date_local' : 'date', 'arithmetic_mean' : 'temp'})
union_temp2['year'] = union_temp2['date'].dt.year
union_temp2['month'] = union_temp2['date'].dt.month
union_temp2['year_month'] = union_temp2['month'].map(str) + '-' + union_temp2['year'].map(str)
union_temp2['year_month'] = pd.to_datetime(union_temp2['year_month'], format='%m-%Y')
union_temp2['months_since'] = ((union_temp2['date'] - union_temp2['date'][0])/np.timedelta64(1, 'M')).astype(int)
display(union_temp2)

,date,temp,cbsa_name,year,month,year_month,months_since
0,1980-01-01,47.625000,"Huntsville, AL",1980,1,1980-01-01,0
1,1980-01-02,58.727273,"Huntsville, AL",1980,1,1980-01-01,0
2,1980-01-03,56.625000,"Huntsville, AL",1980,1,1980-01-01,0
3,1980-01-04,47.875000,"Huntsville, AL",1980,1,1980-01-01,0
4,1980-01-05,41.125000,"Huntsville, AL",1980,1,1980-01-01,0
...,...,...,...,...,...,...,...
1588177,1999-12-27,22.625000,"Jackson, WY-ID",1999,12,1999-12-01,239
1588178,1999-12-28,21.250000,"Jackson, WY-ID",1999,12,1999-12-01,239
1588179,1999-12-29,17.041667,"Jackson, WY-ID",1999,12,1999-12-01,239
1588180,1999-12-30,15.791667,"Jackson, WY-ID",1999,12,1999-12-01,239


In [8]:
union_aqi.to_csv('union_aqi.csv')
test_aqi.to_csv('test_aqi.csv')
union_temp.to_csv('union_temp.csv')
test_temp.to_csv('test_temp.csv')
union_temp2.to_csv('union_temp2.csv')